In [1]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, Trainer, DataCollatorForTokenClassification, TrainingArguments, Trainer
import datasets
import evaluate

import numpy as np

from huggingface_hub import notebook_login

2023-03-03 12:43:58.872725: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2023-03-03 12:43:58.872747: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
data = datasets.load_dataset('RCC-MSU/collection3')
ner_feature = data["train"].features["ner_tags"]
label_names = ner_feature.feature.names
label_names

Found cached dataset collection3 (/home/viktor-sch/.cache/huggingface/datasets/collection3/default/1.0.0/76e3d2db83e2e11c3f99257b9616e9704217d691d6c9ff575e021e1ca1569311)


  0%|          | 0/3 [00:00<?, ?it/s]

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

In [3]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [4]:
model = AutoModelForTokenClassification.from_pretrained('sberbank-ai/ruBert-base', id2label=id2label, label2id=label2id)
tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/ruBert-base')

Some weights of the model checkpoint at sberbank-ai/ruBert-base were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized fr

In [5]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

tokenized_datasets = data.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=data["train"].column_names,
)

Loading cached processed dataset at /home/viktor-sch/.cache/huggingface/datasets/collection3/default/1.0.0/76e3d2db83e2e11c3f99257b9616e9704217d691d6c9ff575e021e1ca1569311/cache-a1a984ba6c39e742.arrow
Loading cached processed dataset at /home/viktor-sch/.cache/huggingface/datasets/collection3/default/1.0.0/76e3d2db83e2e11c3f99257b9616e9704217d691d6c9ff575e021e1ca1569311/cache-950f0ec6be0995e8.arrow
Loading cached processed dataset at /home/viktor-sch/.cache/huggingface/datasets/collection3/default/1.0.0/76e3d2db83e2e11c3f99257b9616e9704217d691d6c9ff575e021e1ca1569311/cache-55b748fb52744fe5.arrow


In [6]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [7]:
metric = evaluate.load("seqeval")

In [8]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [10]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/viktor-sch/.cache/huggingface/token
Login successful


In [13]:
args = TrainingArguments(
    "sberbank-rubert-base-collection3",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    num_train_epochs=5,
    warmup_steps=0.1,
    lr_scheduler_type="cosine",
    push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [14]:

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

Cloning https://huggingface.co/viktoroo/sberbank-rubert-base-collection3 into local empty directory.
/home/viktor-sch/Clones/collection3-model-attack/venv7/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 9301
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 11630
  Number of trainable parameters = 177722119
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.089900,0.075957,0.903977,0.932961,0.918240,0.978725
2,0.052200,0.067969,0.933026,0.933932,0.933479,0.982127
3,0.025900,0.074482,0.930830,0.951178,0.940894,0.983774
4,0.011400,0.073128,0.937218,0.957250,0.947128,0.985713
5,0.002700,0.077185,0.938019,0.959436,0.948607,0.986042


***** Running Evaluation *****
  Num examples = 2153
  Batch size = 8
Saving model checkpoint to sberbank-rubert-base-collection3/checkpoint-2326
Configuration saved in sberbank-rubert-base-collection3/checkpoint-2326/config.json
Model weights saved in sberbank-rubert-base-collection3/checkpoint-2326/pytorch_model.bin
tokenizer config file saved in sberbank-rubert-base-collection3/checkpoint-2326/tokenizer_config.json
Special tokens file saved in sberbank-rubert-base-collection3/checkpoint-2326/special_tokens_map.json
tokenizer config file saved in sberbank-rubert-base-collection3/tokenizer_config.json
Special tokens file saved in sberbank-rubert-base-collection3/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2153
  Batch size = 8
Saving model checkpoint to sberbank-rubert-base-collection3/checkpoint-4652
Configuration saved in sberbank-rubert-base-collection3/checkpoint-4652/config.json
Model weights saved in sberbank-rubert-base-collection3/checkpoint-4652/py

TrainOutput(global_step=11630, training_loss=0.04120228877891678, metrics={'train_runtime': 1133.1573, 'train_samples_per_second': 41.04, 'train_steps_per_second': 10.263, 'total_flos': 1027100378307762.0, 'train_loss': 0.04120228877891678, 'epoch': 5.0})

In [ ]:
trainer.push_to_hub(commit_message="Training complete")

Saving model checkpoint to sberbank-rubert-base-collection3
Configuration saved in sberbank-rubert-base-collection3/config.json
Model weights saved in sberbank-rubert-base-collection3/pytorch_model.bin
tokenizer config file saved in sberbank-rubert-base-collection3/tokenizer_config.json
Special tokens file saved in sberbank-rubert-base-collection3/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 32.0k/678M [00:00<?, ?B/s]

Upload file runs/Mar03_12-56-54_viktor-sch/events.out.tfevents.1677837429.viktor-sch.21172.0: 100%|##########|…

In [ ]:
trainer.evaluate(tokenized_datasets["test"])